In [ ]:
!pip install --upgrade matplotlib numpy prometheus-api-client scikit-learn torch urllib3

In [ ]:
from copy                        import deepcopy
from datetime                    import timedelta
from matplotlib.pyplot           import legend, plot, show, xlabel, ylabel
from numpy                       import flip, zeros
from prometheus_api_client       import MetricSnapshotDataFrame, PrometheusConnect
from prometheus_api_client.utils import parse_datetime
from urllib3                     import disable_warnings
from torch                       import no_grad, save, tensor, zeros as torch_zeros
from torch.nn                    import Linear, LSTM, Module, MSELoss
from torch.optim                 import Adam
from torch.utils.data            import DataLoader, Dataset
from sklearn.preprocessing       import MinMaxScaler

disable_warnings()

In [ ]:
PROMETHEUS_URL   = '<PROMETHEUS_URL>'
PROMETHEUS_TOKEN = '<PROMETHEUS_TOKEN>'

In [ ]:
prometheus_connect = PrometheusConnect(
    url         = PROMETHEUS_URL,
    headers     = { 'Authorization' : f'bearer { PROMETHEUS_TOKEN }' },
    disable_ssl = True
)

In [ ]:
prometheus_connect.all_metrics()[:10]

In [ ]:
metric_name = 'pod:container_cpu_usage:sum'
start_time  = parse_datetime('30m')
end_time    = parse_datetime('now')
chunk_size  = timedelta(seconds = 30)

label_config = {
    'prometheus' : 'openshift-monitoring/k8s',
    'namespace'  : '<namespace>'
}

metric_data = prometheus_connect.get_metric_range_data(
    metric_name  = metric_name,
    label_config = label_config,
    start_time   = start_time,
    end_time     = end_time,
    chunk_size   = chunk_size
)

metric_data = MetricSnapshotDataFrame(metric_data)
metric_data

In [ ]:
len(metric_data)

In [ ]:
def transform_and_normalize(metric_data, lookback):

    metric_data = deepcopy(metric_data)
    metric_data = metric_data[['timestamp', 'value']]

    for index in range(1, lookback + 1):

        metric_data[f't - {index}'] = metric_data['value'].shift(index)

    metric_data.set_index('timestamp', inplace = True)
    metric_data.dropna(inplace = True)

    return metric_data

In [ ]:
lookback = 4

metric_data = transform_and_normalize(metric_data, lookback)
metric_data

In [ ]:
metric_data = metric_data.to_numpy()
metric_data

In [ ]:
scaler = MinMaxScaler(feature_range = (-1, 1))

metric_data = scaler.fit_transform(metric_data)
metric_data

In [ ]:
X = metric_data[:, 1:]
X = deepcopy(flip(X, axis = 1))

split_index = int(len(X) * 0.75)

X_train = X[:split_index]
X_train = X_train.reshape((-1, lookback, 1))
X_train = tensor(X_train).float()

X_test = X[split_index:]
X_test = X_test.reshape((-1, lookback, 1))
X_test = tensor(X_test).float()

y = metric_data[:, 0]

y_train = y[:split_index]
y_train = y_train.reshape((-1, 1))
y_train = tensor(y_train).float()

y_test = y[split_index:]
y_test = y_test.reshape((-1, 1))
y_test = tensor(y_test).float()

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
class PrometheusDataset(Dataset):

    def __init__(self, X, y):

        self.X = X
        self.y = y

    def __len__(self):

        return len(self.X)

    def __getitem__(self, index):

        return self.X[index], self.y[index]

In [ ]:
train_dataset = PrometheusDataset(X_train, y_train)
test_dataset  = PrometheusDataset(X_test, y_test)

In [ ]:
batch_size = 12

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = False)
test_loader  = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [ ]:
class PrometheusLSTM(Module):

    def __init__(self, input_size, hidden_size, stacked_layers):

        super().__init__()

        self.hidden_size    = hidden_size
        self.stacked_layers = stacked_layers

        self.LSTM   = LSTM(input_size, hidden_size, stacked_layers, batch_first = True)
        self.linear = Linear(hidden_size, 1)

    def forward(self, x):

        batch_size = x.size(0)

        h0 = torch_zeros(self.stacked_layers, batch_size, self.hidden_size).to('cpu')
        c0 = torch_zeros(self.stacked_layers, batch_size, self.hidden_size).to('cpu')

        output, _ = self.LSTM(x, (h0, c0))
        output    = self.linear(output[:, -1, :])

        return output

In [ ]:
model = PrometheusLSTM(1, 4, 1)
model.to('cpu')

In [ ]:
def train():

    model.train(True)

    running_loss = 0.0

    for batch_index, batch in enumerate(train_loader):

        x_batch = batch[0].to('cpu')
        y_batch = batch[1].to('cpu')

        output = model(x_batch)

        loss          = loss_function(output, y_batch)
        running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Batch {0}, Loss: {1:.3f}'.format(batch_index + 1, running_loss / 100))

In [ ]:
def validate():

    model.train(False)

    running_loss = 0.0

    for batch_index, batch in enumerate(test_loader):

        x_batch = batch[0].to('cpu')
        y_batch = batch[1].to('cpu')

        with no_grad():

            output = model(x_batch)

            loss          = loss_function(output, y_batch)
            running_loss += loss.item()

    print('Val Loss: {0:.3f}'.format(running_loss / len(test_loader)))

In [ ]:
learning_rate = 0.1
epochs        = 100
loss_function = MSELoss()
optimizer     = Adam(model.parameters(), lr = learning_rate)

In [ ]:
for epoch in range(epochs):

    print('Epoch {0}'.format(epoch + 1))
    train()
    validate()
    print('**********')

In [ ]:
with no_grad():

    predicted = model(X_train.to('cpu')).to('cpu').numpy()

plot(y_train, label = 'Actual Value')
plot(predicted, label = 'Predicted Value')
xlabel('Time')
ylabel('Value')
legend()
show()

In [ ]:
train_predictions = predicted.flatten()

u       = zeros((X_train.shape[0], lookback + 1))
u[:, 0] = train_predictions
u       = scaler.inverse_transform(u)

train_predictions = deepcopy(u[:, 0])

u       = zeros((X_train.shape[0], lookback + 1))
u[:, 0] = y_train.flatten()
u       = scaler.inverse_transform(u)

u_y_train = deepcopy(u[:, 0])

plot(u_y_train, label = 'Actual Value')
plot(train_predictions, label = 'Predicted Value')
xlabel('Time')
ylabel('Value')
legend()
show()

In [ ]:
test_predictions = model(X_test.to('cpu')).detach().cpu().numpy().flatten()

u       = zeros((X_test.shape[0], lookback + 1))
u[:, 0] = test_predictions
u       = scaler.inverse_transform(u)

test_predictions = deepcopy(u[:, 0])

u       = zeros((X_test.shape[0], lookback + 1))
u[:, 0] = y_test.flatten()
u       = scaler.inverse_transform(u)

u_y_test = deepcopy(u[:, 0])

plot(u_y_test, label = 'Actual Value')
plot(test_predictions, label = 'Predicted Value')
xlabel('Time')
ylabel('Value')
legend()
show()

In [ ]:
save(model.state_dict(), '../app/model.pt')